## Upload to Model Registry

In [ ]:
import mlflow
import mlflow.pytorch

In [ ]:
## Log local artifact to experiment

import sys
import torch
sys.path.append('../')

import torch.nn as nn
import torch.optim as optim
from torchvision import models
import numpy as np
import torch.nn.functional as F

class MobileNetV2(nn.Module):
    def __init__(self, num_classes=30):
        super(MobileNetV2, self).__init__()
        self.model = models.mobilenet_v2(pretrained=True)
        self.model.classifier[1] = nn.Linear(self.model.last_channel, num_classes)

    def forward(self, x):
        return self.model(x)

mlflow.set_tracking_uri("http://localhost:5000")

model = MobileNetV2()
model.load_state_dict(torch.load('./artifacts/model_v_2_1_epoch_16.pt', weights_only=True, map_location='cpu'))

with mlflow.start_run():
    mlflow.pytorch.log_model(model, "model")

In [ ]:
import mlflow

mlflow.set_tracking_uri("http://localhost:5000")

model_uri = f"runs:/c43d174ed24849fcacaf931dda5605c8/model"
model_name = "model"

mlflow.register_model(model_uri, model_name)

## Upload updated code to S3

In [ ]:
import sys
import os

sys.path.append('../')

from utils.utils import zip_folder, upload_to_s3

version = 1
folder_to_zip = "../modules"
zip_file_name = "ray_modules"
s3_bucket = "makanlah-s3-bucket" 
s3_key = f"{zip_file_name}-{version}.zip"

zip_file_path = zip_folder(folder_to_zip, zip_file_name)

upload_to_s3(zip_file_path, s3_bucket, s3_key)

## Upload data to S3

In [ ]:
import os
import awswrangler as wr

import os
import awswrangler as wr
from concurrent.futures import ThreadPoolExecutor

def upload_file(local_file_path, s3_file_path):
    wr.s3.upload(local_file_path, s3_file_path)

def upload_folder_to_s3_parallel(local_folder_path, s3_bucket_path, max_workers=10):
    futures = []
    
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        for root, dirs, files in os.walk(local_folder_path):
            for file in files:
                local_file_path = os.path.join(root, file)
                
                relative_file_path = os.path.relpath(local_file_path, local_folder_path)
                
                s3_file_path = os.path.join(s3_bucket_path, relative_file_path).replace("\\", "/")
                
                futures.append(executor.submit(upload_file, local_file_path, s3_file_path))
        
        for future in futures:
            future.result()

In [ ]:
local_folder_path = '../data/in'
s3_bucket_path = 's3://makanlah-s3-bucket/datasets/in'

upload_folder_to_s3_parallel(local_folder_path, s3_bucket_path)

In [ ]:
local_folder_path = '../data/out/screens'
s3_bucket_path = 's3://makanlah-s3-bucket/datasets/out/screens'

upload_folder_to_s3_parallel(local_folder_path, s3_bucket_path)

In [ ]:
upload_file('../data/train_outliers.json', 's3://makanlah-s3-bucket/datasets/train_outliers.json')

In [ ]:
upload_file('../data/test_outliers.json', 's3://makanlah-s3-bucket/datasets/test_outliers.json')

## Sync local data

In [ ]:
import os
import numpy as np
from PIL import Image, ImageOps
import json
import boto3
from io import BytesIO
import botocore.exceptions

s3_client = boto3.client('s3')

def is_s3_path(path):
    return path.lower().startswith("s3://")
    
def split_name_and_key(path):
    return path.replace("s3://", "").split("/", 1)

def read_json(path):
    if is_s3_path(path):
        bucket_name, object_key = split_name_and_key(path)
        obj = s3_client.get_object(Bucket=bucket_name, Key=object_key)
        return json.loads(obj['Body'].read().decode('utf-8'))
    else:
        with open(path, 'r') as f:
            return json.load(f)

def read_image(path):
    if is_s3_path(path):
        bucket_name, object_key = split_name_and_key(path)
        obj = s3_client.get_object(Bucket=bucket_name, Key=object_key)
        img_data = obj['Body'].read()
        img = Image.open(BytesIO(img_data)).convert('RGB')
        return img
    else:
        img = Image.open(path).convert('RGB')
        return img

In [ ]:
for i in ['train_food.json', 'test_food.json', 'train_outliers.json', 'test_outliers.json']:
    s3_uri = 's3://makanlah-s3-bucket/datasets/' + i
    
    s3_dict = read_json(s3_uri)
    
    with open('../data/' + i, 'w') as json_file:
        json.dump(s3_dict, json_file, indent=4)
    
    for i in s3_dict:
        if not os.path.exists('../data/' + i['path']):
            img = read_image('s3://makanlah-s3-bucket/datasets/' + i['path'])
            img.save('../data/' + i['path'])